MaxCut on an undirected graph $G = \{V, E\}$ with $n = 4$ vertices $V = \{0, 1, 2, 3\}$ and 4 edges $E = \{(0,1), (0,3), (1,2), (2,3)\}$ with unit weight $w_{i, j} = 1$.
The MaxCut classical objective function is as follows
$$
L(z) = \sum_{\{i, j\} \in E} w_{i, j}z_i(1 - z_j) + w_{i, j} z_j(1 - z_i),
$$
with $z_i \in \{-1, 1\}$ for $i \in V$.

The equivalent problem Hamiltonian can be described in terms of $Z$ interactions:
$$
H_L = \frac{1}{2} \sum_{\{i, j\} \in E} w_{i, j}(I - Z^{(i)}Z^{(j)}),
$$
where $Z^{(i)}$ is the Pauli-Z operator acting on qubit $i$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'''
\usepackage{palatino}
\usepackage{newpxmath}''')

a = '#bdc3c7'

n = 4
V = np.arange(0, n, 1)
E =[(0, 1, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)] 

G = nx.Graph()
G.add_nodes_from(V)
G.add_weighted_edges_from(E)

edge_labels = nx.get_edge_attributes(G, 'weight')
pos = nx.spring_layout(G, seed=5)

plt.subplots(figsize=(4, 3), dpi=190)
default_axes = plt.axes(frameon=False)

nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)
nx.draw_networkx(
    G, node_size=1000, alpha=1, ax=default_axes, pos=pos,
    width=3, font_color='#ffffff',
    node_color=[a for _ in V],
)

plt.axis('off')
axis = plt.gca()
axis.set_xlim([1.1*x for x in axis.get_xlim()])
axis.set_ylim([1.1*y for y in axis.get_ylim()])
plt.tight_layout()
plt.savefig('visualizations/figures/maxcut_4_graph.pdf', bbox_inches='tight')
plt.show()

/home/soud/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


<Figure size 760x570 with 1 Axes>

In [2]:
def L(G, z):
    cost = 0
    
    for i, j in G.edges:
        w = G[i][j]['weight']
        cost += w*z[i]*(1 - z[j]) + w*z[j]*(1 - z[i])
    
    return cost

In [3]:
import itertools

zs = list(itertools.product([0, 1], repeat=4))
for z in zs:
    print(f'L({z}) = {L(G, z)}')

L((0, 0, 0, 0)) = 0.0
L((0, 0, 0, 1)) = 2.0
L((0, 0, 1, 0)) = 2.0
L((0, 0, 1, 1)) = 2.0
L((0, 1, 0, 0)) = 2.0
L((0, 1, 0, 1)) = 4.0
L((0, 1, 1, 0)) = 2.0
L((0, 1, 1, 1)) = 2.0
L((1, 0, 0, 0)) = 2.0
L((1, 0, 0, 1)) = 2.0
L((1, 0, 1, 0)) = 4.0
L((1, 0, 1, 1)) = 2.0
L((1, 1, 0, 0)) = 2.0
L((1, 1, 0, 1)) = 2.0
L((1, 1, 1, 0)) = 2.0
L((1, 1, 1, 1)) = 0.0


Optimal solutions are $z = 1010$ and $z = 0101$ with $L(z) =  4$.

In [4]:
from projectq import MainEngine
from projectq.backends import ResourceCounter, Simulator
from projectq.ops import All, H, X, Rz, Rx, Ry, CNOT, Measure, QubitOperator
from projectq.setups import restrictedgateset
from quantuminspire.projectq.backend_qx import QIBackend

class QAOA:
    def __init__(self, G, n, p=1, shots=1024,
                 eng_kwargs={}, use_qi=False, qi_backend_kwargs=None):
        self.use_qi = use_qi
        self.eng_kwargs = eng_kwargs
        self.qi_backend_kwargs = qi_backend_kwargs
        self._renew_engine()
            
        self.G = G
        self.p = p
        self.shots = shots
        
    def _renew_engine(self):
        if self.use_qi:
            if self.qi_backend_kwargs is None:
                raise RuntimeError("qi_backend_kwargs is required when use_qi=True")
            qi_backend = QIBackend(num_runs=self.shots, **self.qi_backend_kwargs)
            
            compiler_engines = restrictedgateset.get_engine_list(
                one_qubit_gates=qi_backend.one_qubit_gates,
                two_qubit_gates=qi_backend.two_qubit_gates,
                other_gates=qi_backend.three_qubit_gates
            )
            compiler_engines.extend([ResourceCounter()])
            
            self.eng_kwargs["backend"] = qi_backend
            self.eng_kwargs["engine_list"] = compiler_engines
        else:
            self.eng_kwargs["backend"] = Simulator()
            

        self.eng = MainEngine(**self.eng_kwargs)
        
    def cost(self, params):
        print(f'Params: {params}')
        cost = 0
        cost_hamiltonian = 0 * QubitOperator('')
        for i, j in G.edges:
            w = G[i][j]['weight']
            cost_hamiltonian += 0.5 * w * (QubitOperator('') - QubitOperator(f"Z{i} Z{j}"))

        cost -= self.expval(cost_hamiltonian, params)
        
        print(f'Objective: {-cost:.5f}')

        return cost
        
    def qaoa_circuit(self, params):
        qureg = self.eng.allocate_qureg(n)
        
        All(H) | qureg

        for i in range(self.p):
            # problem unitary
            for k, l in self.G.edges:
                CNOT | (qureg[k], qureg[l])
                Rz(params[i]*G[k][l]['weight']) | qureg[l]
                CNOT | (qureg[k], qureg[l])

            # mixer unitary
            All(Rx(2*params[i+1])) | qureg
        
        return qureg

    @staticmethod
    def _even_ones(binary, relevant_idxs):
        integer = int(binary)
        ones_rel_idxs = 0
        for i in relevant_idxs:
            ones_rel_idxs += 10**i
        
        n_ones = str(integer + ones_rel_idxs).count('2')
        
        return n_ones % 2 == 0
    
    def _get_qi_probs(self, qureg):
        results = {}
        probs = self.eng.backend.get_probabilities(qureg)
        
        for result, count in probs.items():
            reversed_order = ''.join(list(reversed(result)))
            results[reversed_order] = count
        
        return results
    
    def probabilities(self, params):
        self._renew_engine()
        
        if self.use_qi:
            # Quantum Inspire back-end does 1024 shots by default and
            #   saves probabilities in the back-end object
            qureg = self.qaoa_circuit(params)
            All(Measure) | qureg

            self.eng.flush()
            results = self._get_qi_probs(qureg)
            self._renew_engine()

            return results
        else:
            # sample results manually through repeated shots
            results = {}
            for _ in range(self.shots): 
                qureg = self.qaoa_circuit(params)
                All(Measure) | qureg

                result = ''.join(str(int(q)) for q in reversed(qureg))
                results[result] = results.get(result, 0) + 1

                self.eng.flush(deallocate_qubits=True)

            for result, count in results.items():
                results[result] = count/self.shots

            return results
    
    def expval(self, operator, params):
        expval = 0
        
        for term, coef in operator.terms.items():
            rotations = []
            qubits_of_interest = []
            
            # if term is identity, add constant coefficient
            if term == (): 
                expval += coef
                continue
            
            for qubit, op in term:     
                qubits_of_interest.append(qubit)
                
                if op == 'X':
                    rotations.append((Ry(-np.pi/2), qubit))
                elif op == 'Y':
                    rotations.append((Rx(np.pi/2), qubit))
            
            results = None
            if self.use_qi:
                # Quantum Inspire back-end does 1024 shots by default and
                #   saves probabilities in the back-end object
                qureg = self.qaoa_circuit(params)

                for rot, i in reversed(rotations):
                    rot | qureg[i]

                All(Measure) | qureg

                self.eng.flush()
                results = self._get_qi_probs(qureg)
                # can only use engine once with QI back-end
                self._renew_engine() 
            else:
                # sample results manually through repeated shots
                results = {}
                for _ in range(self.shots): 
                    qureg = self.qaoa_circuit(params)

                    for rot, i in reversed(rotations):
                        rot | qureg[i]

                    All(Measure) | qureg

                    result = ''.join(str(int(q)) for q in reversed(qureg))
                    results[result] = results.get(result, 0) + 1
                    
                    self.eng.flush(deallocate_qubits=True)
                
                for result, count in results.items():
                    results[result] = count/self.shots
                
            for result, count in results.items():
                if self._even_ones(result, qubits_of_interest):
                    parity = 1
                else:
                    parity = -1

                expval += parity * coef * count
        
        return expval

In [6]:
def test_hamiltonian(qaoa):
    # expectation value should be -5
    return qaoa.expval(3 * QubitOperator('X0 Z1') + QubitOperator('Y1') + 2 * QubitOperator('Z1'))

In [5]:
from quantuminspire.credentials import enable_account, get_token_authentication
from quantuminspire.api import QuantumInspireAPI
  
enable_account('6ea0bc8530ed4c97073aff535b5b52fd617cc6ed')
auth = get_token_authentication()
server_url = r'https://api.quantum-inspire.com'
qi = QuantumInspireAPI(server_url, auth, project_name='qaoa_maxcut')

np.random.seed(0xc0ffee)
params = np.random.uniform(low=0, high=2*np.pi, size=(2, 1))

qaoa = QAOA(G, n,use_qi=False,
            qi_backend_kwargs={"quantum_inspire_api": qi, "backend_type": "QX single-node simulator"})

In [6]:
from scipy.optimize import minimize

res = minimize(qaoa.cost, params, method='COBYLA',
               bounds=[(0, 2*np.pi) for _ in params],
               options={'disp': True, 'tol': 1e-2, 'maxiter': 15})

/home/soud/anaconda3/lib/python3.7/site-packages/scipy/optimize/_minimize.py:542: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


Params: [3.59968607 5.12202154]
Objective: 1.18457
Params: [4.59968607 5.12202154]
Objective: 1.73926
Params: [4.59968607 6.12202154]
Objective: 2.12988
Params: [5.41729171 6.69780016]
Objective: 2.99316
Params: [6.31161053 7.14523043]
Objective: 1.98535
Params: [5.04660438 7.03334585]
Objective: 2.09277
Params: [5.44469102 6.19855145]
Objective: 1.68750
Params: [5.63420004 6.82210132]
Objective: 2.80469
Params: [5.43986093 6.57485452]
Objective: 2.92871
Params: [5.29796175 6.7350206 ]
Objective: 2.91602
Params: [5.46624608 6.73665528]
Objective: 2.96191
Params: [5.42471105 6.66744368]
Objective: 2.99609
Params: [5.4095328  6.66373402]
Objective: 3.00586
Params: [5.38061276 6.65189367]
Objective: 2.97070
Params: [5.42364796 6.65703304]
Objective: 2.99023


In [7]:
best_params = res.x
probs = qaoa.probabilities(best_params)
highest_prob = [int(b) for b in max(probs, key=probs.get)]
real_objective = L(G, highest_prob)
print(f'Best approximate solution = {highest_prob} with L({highest_prob}) = {real_objective}')

Best approximate solution = [0, 1, 0, 1] with L([0, 1, 0, 1]) = 4.0


In [8]:
from scipy.optimize import minimize

np.random.seed(0xdeadbeef)
params = np.random.uniform(low=0, high=2*np.pi, size=(2, 2))

qaoa = QAOA(G, n, p=2, use_qi=False)

res = minimize(qaoa.cost, params, method='COBYLA',
               bounds=[(0, 2*np.pi) for _ in params],
               options={'disp': True, 'tol': 1e-2, 'maxiter': 15})

Params: [1.39932397 3.9571841  4.85623429 1.37297588]
Objective: 1.52637
Params: [2.39932397 3.9571841  4.85623429 1.37297588]
Objective: 1.41602
Params: [1.39932397 4.9571841  4.85623429 1.37297588]
Objective: 1.70703
Params: [1.39932397 4.9571841  5.85623429 1.37297588]
Objective: 2.20996
Params: [1.39932397 4.9571841  5.85623429 2.37297588]
Objective: 2.21582
Params: [1.19710414 5.28825196 6.77785565 2.38371322]
Objective: 2.95801
Params: [1.06826778 5.11073023 7.75347875 2.39055409]
Objective: 1.17871
Params: [1.6845627  5.3669417  6.85654539 2.38371322]
Objective: 3.48340
Params: [1.76886675 6.34814433 6.68295717 2.38488027]
Objective: 2.47363
Params: [2.21509911 4.90252835 7.56565874 2.38681214]
Objective: 3.34277
Params: [1.84900227 5.21346523 6.61290571 2.75793817]
Objective: 3.43945
Params: [1.86863324 5.19599721 6.58755258 2.04527727]
Objective: 3.32129
Params: [1.80184215 5.26501334 7.05145454 2.40285905]
Objective: 3.57129
Params: [1.9621554  5.43899894 7.12712875 2.4311802

In [13]:
best_params = res.x.flatten('F')
probs = qaoa.probabilities(best_params)
highest_prob = [int(b) for b in max(probs, key=probs.get)]
real_objective = L(G, highest_prob)
print(f'Best approximate solution = {highest_prob} with L({highest_prob}) = {real_objective}')

Best approximate solution = [1, 0, 1, 0] with L([1, 0, 1, 0]) = 4.0


In [14]:
res

     fun: -3.5712890625
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 15
  status: 2
 success: False
       x: array([[1.80184215, 7.05145454],
       [5.26501334, 2.40285905]])